### Load tensorflow

In [0]:
import tensorflow as tf

### Enable Eager Execution if you are using tensorflow 1.x

In [0]:
tf.enable_eager_execution()

### Collect Data

In [0]:
import pandas as pd
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/prices.csv')

### Check all columns in the dataset

In [6]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns = {'date','symbol'},inplace = True)

In [8]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [9]:
data.shape

(851264, 5)

In [0]:
dataset = data.iloc[:1000,:]

In [11]:
dataset.shape

(1000, 5)

### Convert Float64 to Float32

In [12]:
dataset = dataset.astype('float32')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
open      1000 non-null float32
close     1000 non-null float32
low       1000 non-null float32
high      1000 non-null float32
volume    1000 non-null float32
dtypes: float32(5)
memory usage: 19.6 KB


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split 

In [0]:
y = dataset['close']

In [0]:
X = dataset.drop(columns = 'close')

In [0]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.7)

### Normalize Train and Test Data 

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
norm = Normalizer()

In [0]:
train_x = norm.fit_transform(X_train)

## Building the graph in tensorflow

2.Define Weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [27]:
for i in range(100):
    
    w, b = train(train_x, np.array(y_train), w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, loss(np.array(y_train), prediction(train_x, w, b)).numpy())

Current Loss on iteration 0 8212.707
Current Loss on iteration 1 7881.321
Current Loss on iteration 2 7575.9155
Current Loss on iteration 3 7294.4526
Current Loss on iteration 4 7035.0576
Current Loss on iteration 5 6795.9995
Current Loss on iteration 6 6575.6816
Current Loss on iteration 7 6372.6387
Current Loss on iteration 8 6185.5137
Current Loss on iteration 9 6013.0596
Current Loss on iteration 10 5854.1255
Current Loss on iteration 11 5707.652
Current Loss on iteration 12 5572.6616
Current Loss on iteration 13 5448.255
Current Loss on iteration 14 5333.601
Current Loss on iteration 15 5227.937
Current Loss on iteration 16 5130.5566
Current Loss on iteration 17 5040.8105
Current Loss on iteration 18 4958.1006
Current Loss on iteration 19 4881.875
Current Loss on iteration 20 4811.6255
Current Loss on iteration 21 4746.885
Current Loss on iteration 22 4687.2183
Current Loss on iteration 23 4632.23
Current Loss on iteration 24 4581.5537
Current Loss on iteration 25 4534.849
Current

### Get the shapes and values of W and b

In [28]:
print('Weights:\n', w.numpy())

Weights:
 [[2.4683157e-03]
 [2.4446330e-03]
 [2.4887603e-03]
 [3.3290398e+01]]


In [29]:
print('Bias:\n',b.numpy())

Bias:
 [33.290398]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [31]:
model.fit(train_x, np.array(y_train), epochs=100)

Epoch 1/100
700/700 [==============================] - 0s 252us/sample - loss: 6186.7477
Epoch 2/100
700/700 [==============================] - 0s 59us/sample - loss: 4353.9785
Epoch 3/100
700/700 [==============================] - 0s 54us/sample - loss: 4045.8418
Epoch 4/100
700/700 [==============================] - 0s 52us/sample - loss: 4000.0145
Epoch 5/100
700/700 [==============================] - 0s 53us/sample - loss: 3990.3302
Epoch 6/100
700/700 [==============================] - 0s 54us/sample - loss: 3991.6112
Epoch 7/100
700/700 [==============================] - 0s 55us/sample - loss: 3990.6915
Epoch 8/100
700/700 [==============================] - 0s 51us/sample - loss: 3989.9782
Epoch 9/100
700/700 [==============================] - 0s 58us/sample - loss: 3990.0169
Epoch 10/100
700/700 [==============================] - 0s 56us/sample - loss: 3993.2594
Epoch 11/100
700/700 [==============================] - 0s 51us/sample - loss: 3990.5008
Epoch 12/100
700/700 [=======

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [36]:
I_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Iris.csv')
I_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [0]:
X = I_data.drop(columns = 'Species')
y = I_data['Species']

In [39]:
X.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,1,5.1,3.5,1.4,0.2
1,2,4.9,3.0,1.4,0.2
2,3,4.7,3.2,1.3,0.2
3,4,4.6,3.1,1.5,0.2
4,5,5.0,3.6,1.4,0.2


In [0]:
X.drop(columns = 'Id', inplace = True)

In [41]:
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
enc = LabelEncoder()

In [0]:
y = enc.fit_transform(y)

In [45]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [0]:
y_cat = tf.keras.utils.to_categorical(y, num_classes=3)

In [50]:
y_cat[0]

array([1., 0., 0.], dtype=float32)

### Divide the dataset into Training and test (70:30)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=.25, random_state=10)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))

#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [60]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=150, batch_size=9)

Train on 112 samples, validate on 38 samples
Epoch 1/150
112/112 [==============================] - 0s 2ms/sample - loss: 0.4324 - acc: 0.8393 - val_loss: 0.4268 - val_acc: 0.8421
Epoch 2/150
112/112 [==============================] - 0s 499us/sample - loss: 0.4722 - acc: 0.8304 - val_loss: 0.4268 - val_acc: 0.8421
Epoch 3/150
112/112 [==============================] - 0s 413us/sample - loss: 0.4116 - acc: 0.8839 - val_loss: 0.4269 - val_acc: 0.8421
Epoch 4/150
112/112 [==============================] - 0s 390us/sample - loss: 0.4373 - acc: 0.8304 - val_loss: 0.4269 - val_acc: 0.8421
Epoch 5/150
112/112 [==============================] - 0s 391us/sample - loss: 0.4367 - acc: 0.8571 - val_loss: 0.4271 - val_acc: 0.8158
Epoch 6/150
112/112 [==============================] - 0s 402us/sample - loss: 0.4742 - acc: 0.8214 - val_loss: 0.4269 - val_acc: 0.8158
Epoch 7/150
112/112 [==============================] - 0s 407us/sample - loss: 0.4701 - acc: 0.8214 - val_loss: 0.4265 - val_acc: 0.815

In [61]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 4)                 16        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 27        
Total params: 181
Trainable params: 173
Non-trainable params: 8
_________________________________________________________________


### Report Accuracy of the predicted values

In [62]:
pred = model.predict(X_test)
pred

array([[0.01885681, 0.62666905, 0.3544742 ],
       [0.00515039, 0.3180198 , 0.6768298 ],
       [0.94014245, 0.05700803, 0.00284956],
       [0.01923047, 0.5108761 , 0.4698935 ],
       [0.9318728 , 0.06513178, 0.00299548],
       [0.05988625, 0.78641164, 0.15370212],
       [0.01017148, 0.47840068, 0.5114278 ],
       [0.07698244, 0.74552596, 0.17749165],
       [0.9345658 , 0.06250963, 0.00292459],
       [0.0288354 , 0.63674736, 0.33441725],
       [0.02103931, 0.5359303 , 0.4430304 ],
       [0.00609359, 0.36002392, 0.6338825 ],
       [0.02965303, 0.7107431 , 0.25960383],
       [0.9122797 , 0.08389364, 0.00382662],
       [0.94045335, 0.05667431, 0.0028723 ],
       [0.00421082, 0.25929752, 0.7364917 ],
       [0.02844746, 0.4318378 , 0.53971475],
       [0.91789013, 0.07860328, 0.00350656],
       [0.9382326 , 0.05879061, 0.00297678],
       [0.92834014, 0.06854054, 0.00311925],
       [0.00397425, 0.25023103, 0.7457947 ],
       [0.00414138, 0.27229148, 0.7235671 ],
       [0.

In [0]:
pred_val=[]
for val in pred:  
  pred_val.append(np.argmax(val))

In [68]:
pred_val

[1,
 2,
 0,
 1,
 0,
 1,
 2,
 1,
 0,
 1,
 1,
 2,
 1,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 1,
 0,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 0,
 2,
 2]

In [63]:
model.evaluate(X_test,y_test)

38/38 [==============================] - 0s 2ms/sample - loss: 0.3445 - acc: 0.9211


[0.34453511394952474, 0.92105263]